# CAN Bus Database Analyzer

This notebook analyzes data stored in the SQLite database generated by `can_decoder_cli.py`.

In [1]:
# Import required libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set plot style
plt.style.use('ggplot')
sns.set_theme(style="darkgrid")

## Connect to Database and Select Session

In [ ]:
# --- Database and Session Selection --- #

# Find database files
db_files = []
log_dir = 'logs'
if os.path.exists(log_dir):
    db_files = [os.path.join(log_dir, f) for f in os.listdir(log_dir) if f.endswith('.db')]

if not db_files:
    print(f"No database files found in '{log_dir}' directory.")
    db_files = ["No DB Found"]

# Widgets
db_dropdown = widgets.Dropdown(
    options=db_files,
    description='Database File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

session_dropdown = widgets.Dropdown(
    options=["Connect to DB first"],
    description='Session ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'),
    disabled=True
)

connect_button = widgets.Button(description="Connect & List Sessions", button_style='info')
load_session_button = widgets.Button(description="Load Session Data", button_style='primary', disabled=True)
db_output = widgets.Output()

# Global state
db_conn = None
selected_session_id = None
raw_data = None
decoded_data = None

# --- Functions --- #

def get_sessions(conn):
    "Fetches distinct session IDs from the raw_messages table."
    if not conn:
        return []
    try:
        query = "SELECT DISTINCT session_id FROM raw_messages ORDER BY session_id DESC"
        sessions = pd.read_sql_query(query, conn)['session_id'].tolist()
        return sessions
    except Exception as e:
        print(f"Error fetching sessions: {e}")
        # Could be that the table doesn't exist yet or other SQL errors
        if 'no such table: raw_messages' in str(e):
             print("Hint: The 'raw_messages' table was not found. Was the DB created correctly?")
        return []

def close_db_connection():
    "Closes the database connection if it's open."
    global db_conn
    if db_conn:
        print("Closing previous database connection.")
        db_conn.close()
        db_conn = None

def update_downstream_widgets():
    "Updates widgets that depend on loaded data (e.g., parameter selectors)."
    update_parameter_dropdown()
    update_multi_param_selector()
    # Add calls to update other widgets here if necessary

# --- Widget Event Handlers --- #

def on_connect_clicked(b):
    "Handles the 'Connect & List Sessions' button click."
    global db_conn
    with db_output:
        db_output.clear_output()
        close_db_connection() # Close any existing connection first
        
        db_path = db_dropdown.value
        if db_path == "No DB Found" or not os.path.exists(db_path):
            print(f"Database file not selected or not found: {db_path}")
            session_dropdown.options = ["Connect to DB first"]
            session_dropdown.disabled = True
            load_session_button.disabled = True
            return
            
        try:
            print(f"Connecting to {db_path}...")
            db_conn = sqlite3.connect(db_path)
            print(f"Connected successfully.")
            
            sessions = get_sessions(db_conn)
            if sessions:
                session_dropdown.options = sessions
                session_dropdown.disabled = False
                load_session_button.disabled = False
                print(f"Found {len(sessions)} sessions.")
            else:
                print("No sessions found in the database.")
                session_dropdown.options = ["No Sessions Found"]
                session_dropdown.disabled = True
                load_session_button.disabled = True
                
        except Exception as e:
            print(f"Error connecting to database: {e}")
            session_dropdown.options = ["Connection Error"]
            session_dropdown.disabled = True
            load_session_button.disabled = True
            close_db_connection() # Ensure connection is closed on error

def safe_json_loads(x):
    "Safely attempts to decode bytes and parse JSON."
    if isinstance(x, bytes):
        try:
            x = x.decode('utf-8', errors='ignore') # Decode bytes to string
        except Exception:
             return None # Or return original bytes x? 
    if isinstance(x, str):
        try:
            return json.loads(x)
        except json.JSONDecodeError:
            return None # Failed to parse string as JSON
    return x # Assume it's already parsed (e.g., list/dict) or not JSON

def on_load_session_clicked(b):
    "Handles the 'Load Session Data' button click."
    global selected_session_id, raw_data, decoded_data
    with db_output:
        db_output.clear_output(wait=True) # Clear previous output
        selected_session_id = session_dropdown.value
        
        if not db_conn or selected_session_id in ["Connect to DB first", "No Sessions Found", "Connection Error"]:
            print("Please connect to a database and select a valid session.")
            raw_data = None # Reset data if selection is invalid
            decoded_data = None
            update_downstream_widgets() # Update widgets to reflect no data
            return
            
        print(f"Loading data for session: {selected_session_id}...")
        try:
            # --- Load Raw Data --- 
            query_raw = f"SELECT * FROM raw_messages WHERE session_id = ? ORDER BY timestamp ASC"
            raw_data = pd.read_sql_query(query_raw, db_conn, params=(selected_session_id,))
            print(f"Loaded {len(raw_data)} raw messages.")
            
            # Parse 'data' column (potentially bytes or JSON string) into 'data_bytes' list
            if 'data' in raw_data.columns:
                raw_data['data_bytes'] = raw_data['data'].apply(safe_json_loads)
                # Optional: Check how many failed parsing
                parse_failures = raw_data['data_bytes'].isnull().sum()
                if parse_failures > 0 and len(raw_data) > 0: # Avoid division by zero
                     print(f"Warning: Failed to parse 'data' column for {parse_failures} / {len(raw_data)} raw messages.")
            else:
                 print("Warning: 'data' column not found in raw_messages table.")
                 raw_data['data_bytes'] = pd.Series([None] * len(raw_data)) # Ensure column exists

            # --- Load Decoded Data --- 
            query_decoded = f"SELECT * FROM decoded_values WHERE session_id = ? ORDER BY timestamp ASC"
            decoded_data = pd.read_sql_query(query_decoded, db_conn, params=(selected_session_id,))
            print(f"Loaded {len(decoded_data)} decoded value entries.")

            # --- Post-processing --- 
            # Add relative time column (starting from 0 for the session)
            if not raw_data.empty:
                start_time_raw = raw_data['timestamp'].min()
                raw_data['time'] = raw_data['timestamp'] - start_time_raw
            if not decoded_data.empty:
                # Use the raw data start time if available for consistency, else decoded start time
                start_time = start_time_raw if not raw_data.empty else decoded_data['timestamp'].min()
                decoded_data['time'] = decoded_data['timestamp'] - start_time
                
            print("\n--- Raw Data Preview ---")
            display(raw_data.head())
            print("\n--- Decoded Data Preview ---")
            display(decoded_data.head())
            
            # --- Update other UI elements --- 
            update_downstream_widgets()
            print("\nSession data loaded successfully.")
            
        except Exception as e:
            print(f"Error loading session data: {e}")
            # Check for common table errors
            if 'no such table: raw_messages' in str(e) or 'no such table: decoded_values' in str(e):
                 print("Hint: Make sure the database tables 'raw_messages' and 'decoded_values' exist.")
            raw_data = None # Reset data on error
            decoded_data = None
            update_downstream_widgets() # Update widgets to reflect no data

# --- Connect Button Handlers --- #
connect_button.on_click(on_connect_clicked)
load_session_button.on_click(on_load_session_clicked)

# --- Display Initial Widgets --- #
display(widgets.VBox([
    widgets.HBox([db_dropdown, connect_button]), 
    widgets.HBox([session_dropdown, load_session_button]),
    db_output
]))


## Session Overview & Raw Message Statistics

In [ ]:
# --- Session Statistics --- #

stats_button = widgets.Button(description="Show Session Stats", button_style='info')
stats_output = widgets.Output()

def display_session_stats(raw_df, decoded_df):
    "Displays statistics about the loaded session data."
    if raw_df is None or raw_df.empty:
        print("No raw data loaded for the selected session.")
        return
        
    print(f"--- Statistics for Session: {selected_session_id} ---")
    duration = raw_df['time'].max() if 'time' in raw_df.columns and not raw_df.empty else 0
    print(f"Duration: {duration:.2f} seconds")
    print(f"Total Raw Messages: {len(raw_df)}")
    if duration > 0:
        print(f"Average Raw Message Rate: {len(raw_df)/duration:.2f} msgs/sec")
    
    if decoded_df is not None and not decoded_df.empty:
        print(f"Total Decoded Entries: {len(decoded_df)}")
        print(f"Unique Decoded Parameters: {decoded_df['parameter'].nunique()}")
    else:
         print("No decoded data entries found for this session.")
    
    # Raw message counts by CAN ID
    if 'can_id_hex' in raw_df.columns:
        id_counts = raw_df['can_id_hex'].value_counts()
        print("\nTop 10 Most Frequent CAN IDs (Raw Messages):")
        # Correct renaming: Use the actual column names from value_counts().reset_index()
        # Default names are often 'index' and the original column name ('can_id_hex' here)
        id_counts_df = id_counts.head(10).reset_index()
        # Rename based on expected default column names from reset_index()
        id_counts_df = id_counts_df.rename(columns={'index': 'CAN ID Hex', 'can_id_hex': 'Count'}) 
        display(id_counts_df)
        
        # Plot distribution
        plt.figure(figsize=(12, 6))
        top_ids = id_counts.head(15)
        plt.bar(top_ids.index, top_ids.values)
        plt.title('Raw CAN Message Distribution by ID (Top 15)')
        plt.xlabel('CAN ID (hex)')
        plt.ylabel('Number of Messages')
        plt.xticks(rotation=45, ha='right') # Adjust rotation alignment
        plt.tight_layout()
        plt.show()
    else:
        print("\n'can_id_hex' column not found in raw data.")

def on_stats_clicked(b):
    "Handles the 'Show Session Stats' button click."
    with stats_output:
        stats_output.clear_output()
        if raw_data is None:
            print("Please load session data first.")
            return
        display_session_stats(raw_data, decoded_data)
        
# --- Connect Handler & Display --- #
stats_button.on_click(on_stats_clicked)
display(stats_button)
display(stats_output)

Button(button_style='info', description='Show Session Stats', style=ButtonStyle())

Output()

## Analyze Decoded Parameters (Single)

In [4]:
# --- Single Parameter Analysis --- #

# Widgets
param_dropdown = widgets.Dropdown(
    options=["Load session data first"],
    description='Parameter:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

plot_param_button = widgets.Button(description="Plot Parameter", button_style='success')
param_output = widgets.Output()

# --- Functions --- #
def update_parameter_dropdown():
    "Updates the single parameter dropdown based on loaded decoded_data."
    if decoded_data is None or decoded_data.empty:
        param_dropdown.options = ["No decoded data"]
        param_dropdown.value = "No decoded data"
    else:
        params = sorted(decoded_data['parameter'].unique().tolist())
        param_dropdown.options = params
        if params: # Set a default value if list is not empty
             param_dropdown.value = params[0]
        else:
             param_dropdown.options = ["No parameters found"]
             param_dropdown.value = "No parameters found"

def plot_parameter(df, parameter_name):
    "Plots a single decoded parameter over time."
    if df is None or df.empty:
        print("No decoded data available.")
        return
        
    param_df = df[df['parameter'] == parameter_name]
    if param_df.empty:
        print(f"No data found for parameter: {parameter_name}")
        return
        
    # Check if data is numeric
    if not pd.api.types.is_numeric_dtype(param_df['value']):
        print(f"Parameter '{parameter_name}' is not numeric and cannot be plotted.")
        # Optionally display unique values for non-numeric data
        print(f"Unique values: {param_df['value'].unique()}")
        return
        
    # Basic Stats
    print(f"--- Analysis for Parameter: {parameter_name} ---")
    print(f"Number of data points: {len(param_df)}")
    print(f"Min value: {param_df['value'].min():.2f}")
    print(f"Max value: {param_df['value'].max():.2f}")
    print(f"Average value: {param_df['value'].mean():.2f}")
    print(f"Standard deviation: {param_df['value'].std():.2f}")
    
    # Plotting
    plt.figure(figsize=(14, 7))
    plt.plot(param_df['time'], param_df['value'], 'o-', markersize=3, label=parameter_name)
    plt.title(f'{parameter_name} Over Time (Session: {selected_session_id})')
    plt.xlabel('Time (s)')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# --- Widget Event Handler --- #
def on_plot_param_clicked(b):
    "Handles the 'Plot Parameter' button click."
    with param_output:
        param_output.clear_output()
        selected_param = param_dropdown.value
        if decoded_data is None or selected_param in ["Load session data first", "No decoded data", "No parameters found"]:
            print("Please load session data and select a valid parameter.")
            return
        plot_parameter(decoded_data, selected_param)

# --- Connect Handler & Display --- #
plot_param_button.on_click(on_plot_param_clicked)
display(widgets.HBox([param_dropdown, plot_param_button]))
display(param_output)

# Initial update in case data is already loaded from a previous run
update_parameter_dropdown()

Output()

## Analyze Decoded Parameters (Multiple)

In [5]:
# --- Multi-Parameter Analysis --- #

# Widgets
multi_param_selector = widgets.SelectMultiple(
    options=['Load session data first'],
    description='Parameters:',
    disabled=False,
    layout=widgets.Layout(height='150px', width='400px') # Adjust size
)

plot_multi_button = widgets.Button(description="Plot Selected Parameters", button_style='success')
multi_param_output = widgets.Output()

# --- Functions --- #
def update_multi_param_selector():
    "Updates the multi-parameter selector based on loaded decoded_data."
    if decoded_data is None or decoded_data.empty:
        multi_param_selector.options = ["No decoded data"]
        multi_param_selector.value = [] # Ensure value is empty list
    else:
        # Filter for numeric parameters only for plotting suitability
        numeric_params = []
        for param in decoded_data['parameter'].unique():
            # Check dtype of the 'value' column for this parameter
            if pd.api.types.is_numeric_dtype(decoded_data.loc[decoded_data['parameter'] == param, 'value']):
                 numeric_params.append(param)
        
        params = sorted(numeric_params)
        multi_param_selector.options = params
        multi_param_selector.value = [] # Reset selection on update
        if not params:
             multi_param_selector.options = ["No numeric parameters found"]

def plot_multiple_parameters(df, selected_parameters):
    "Plots multiple selected decoded parameters on the same axes."
    if df is None or df.empty:
        print("No decoded data available.")
        return
    if not selected_parameters:
        print("Please select at least one parameter to plot.")
        return
        
    fig, ax1 = plt.subplots(figsize=(14, 7))
    # Use a color map that provides distinct colors
    colors = plt.cm.get_cmap('tab10', len(selected_parameters)) 
    
    print(f"--- Plotting Parameters: {', '.join(selected_parameters)} ---")
    
    plotted_params = []
    for i, param_name in enumerate(selected_parameters):
        param_df = df[df['parameter'] == param_name]
        if not param_df.empty:
            # Double-check if data is numeric before plotting (should be pre-filtered by selector)
            if pd.api.types.is_numeric_dtype(param_df['value']):
                 ax1.plot(param_df['time'], param_df['value'], 'o-', markersize=2, color=colors(i), label=param_name)
                 plotted_params.append(param_name)
            else:
                 # This case should ideally not happen if selector is filtered
                 print(f"Skipping non-numeric parameter: {param_name}")
        else:
            print(f"No data found for parameter: {param_name}")
    
    if not plotted_params:
        print("No numeric data found for any of the selected parameters.")
        plt.close(fig) # Close the empty figure
        return
        
    ax1.set_title(f'Selected Parameters Over Time (Session: {selected_session_id})')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Value')
    # Improve legend positioning if many items
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
    ax1.grid(True)
    # Adjust layout to prevent legend overlap
    plt.tight_layout(rect=[0, 0, 0.85, 1]) # Leave space on the right for legend
    plt.show()

# --- Widget Event Handler --- #
def on_plot_multi_clicked(b):
    "Handles the 'Plot Selected Parameters' button click."
    with multi_param_output:
        multi_param_output.clear_output()
        selected_params = multi_param_selector.value
        # Check if the list is empty or contains placeholder text
        if decoded_data is None or not selected_params or selected_params[0] in ["Load session data first", "No decoded data", "No numeric parameters found"]:
            print("Please load session data and select valid numeric parameters.")
            return
        plot_multiple_parameters(decoded_data, list(selected_params))

# --- Connect Handler & Display --- #
plot_multi_button.on_click(on_plot_multi_clicked)
display(widgets.VBox([
    widgets.Label("Select Numeric Parameters to Plot:"), # Add label for clarity
    multi_param_selector, 
    plot_multi_button
    ]))
display(multi_param_output)

# Initial update in case data is already loaded
update_multi_param_selector()

Output()

## Database Connection Cleanup

In [6]:
# --- Database Cleanup --- #

# Make sure to close the DB connection when done 
# (e.g., at the end of the notebook, or when connecting to a new DB).
# The close_db_connection() function is defined in the first code cell.

# Example of explicitly calling it (you might put this in a final cell):
# close_db_connection() 

# Optional: Register it to run automatically when the kernel shuts down 
# (Note: This might not always work reliably in all Jupyter environments).
import atexit
atexit.register(close_db_connection)

print("Database cleanup function registered.")

Database cleanup function registered.
